<a href="https://colab.research.google.com/github/satyakisen/pandas-ff-comparison/blob/main/Pandas_File_Format_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas File Format Comparision
## Overview
In this notebook let us compare the following pandas file formats.
1. csv - common text file that is comma seperated.
2. hdf5 - an open source file format that supports large, complex, heterogeneous data
3. parquet - an open source, column-oriented data file format designed for efficient data storage and retrieval.
4. feather - a portable file format for storing Arrow tables or data frames (from languages like Python or R) that utilizes the Arrow IPC format internally

Comparision parameters to consider are:
1. Time to write.
2. Time to read.
3. File size on disk.
4. Memory Usage.


## Testing with Numerical Data
Let us begin by creating a dummy dataset containing only random float.

In [1]:
import pandas as pd
import numpy as np

def make_data(row_n, col_n):
  arr = np.random.randn(row_n, col_n)
  df = pd.DataFrame(arr, columns=['col_{0}'.format(i) for i in range(col_n)])
  return df

df = make_data(100000, 10)

Let us check the dummy dataset we made.

In [2]:
df.head(5)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
0,1.032343,-0.197297,-1.608835,-0.715695,0.068162,-0.110879,0.663716,0.063649,-0.727138,1.636330
1,-0.113867,1.376240,-0.805404,2.744988,-1.671231,0.984887,0.526337,-0.324261,0.113425,1.685077
2,0.302444,1.451256,-0.672890,1.029487,0.289066,0.553913,0.367815,0.775710,0.139716,1.289444
3,1.688790,0.331420,-0.362144,0.795289,-0.732524,-0.605117,0.993917,0.946879,-0.171658,-0.531674
4,1.276149,0.243715,1.680892,1.786604,0.810132,-0.060224,-1.982808,0.244423,0.088049,-0.238936


### Time to write

Let us now check the writing time of the above dataframe we created. We will first create a decorator for calculating time and then write a function to save the dataframe into different file formats.

In [3]:
%time df.to_csv('test.csv', index=False)

CPU times: user 1.53 s, sys: 53.9 ms, total: 1.59 s
Wall time: 1.59 s


In [4]:
%time df.to_hdf('test.h5', key='root')

CPU times: user 48.3 ms, sys: 18.3 ms, total: 66.6 ms
Wall time: 182 ms


In [5]:
%time df.to_parquet('test.parquet')

CPU times: user 129 ms, sys: 24.6 ms, total: 154 ms
Wall time: 201 ms


In [6]:
%time df.to_feather('test.feather')

CPU times: user 33.6 ms, sys: 13.2 ms, total: 46.8 ms
Wall time: 44.2 ms


From above we can see that **feather** & **hdf** are the file formats which works pretty well for writing to disk. Now let us consider the second parameter.

### File size on Disk
Let us now check the file size on disk.

In [7]:
%%bash
du -sh test.*

19M	test.csv
7.7M	test.feather
8.5M	test.h5
9.7M	test.parquet


From the above result we can see that **feather** and **hdf** outperforms others. But we will check the performance again with some big data afterward.

### Time to read
Let us check how much time does it take for a read operation.

In [ ]:
%%time 
df_csv=pd.read_csv('test.csv')

CPU times: user 203 ms, sys: 35.9 ms, total: 239 ms
Wall time: 245 ms


In [ ]:
%%time
df_hdf=pd.read_hdf('test.h5')

CPU times: user 16.1 ms, sys: 5.14 ms, total: 21.3 ms
Wall time: 20.6 ms


In [ ]:
%%time
df_parquet=pd.read_parquet('test.parquet')

CPU times: user 23.9 ms, sys: 29.5 ms, total: 53.3 ms
Wall time: 87.3 ms


In [ ]:
%%time
df_feather=pd.read_feather('test.feather')

CPU times: user 10.7 ms, sys: 9.31 ms, total: 20 ms
Wall time: 17.2 ms


### Memory Usage

In [ ]:
df_csv.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   col_0   100000 non-null  float64
 1   col_1   100000 non-null  float64
 2   col_2   100000 non-null  float64
 3   col_3   100000 non-null  float64
 4   col_4   100000 non-null  float64
 5   col_5   100000 non-null  float64
 6   col_6   100000 non-null  float64
 7   col_7   100000 non-null  float64
 8   col_8   100000 non-null  float64
 9   col_9   100000 non-null  float64
dtypes: float64(10)
memory usage: 7.6 MB


In [ ]:
df_hdf.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   col_0   100000 non-null  float64
 1   col_1   100000 non-null  float64
 2   col_2   100000 non-null  float64
 3   col_3   100000 non-null  float64
 4   col_4   100000 non-null  float64
 5   col_5   100000 non-null  float64
 6   col_6   100000 non-null  float64
 7   col_7   100000 non-null  float64
 8   col_8   100000 non-null  float64
 9   col_9   100000 non-null  float64
dtypes: float64(10)
memory usage: 8.4 MB


In [ ]:
df_parquet.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   col_0   100000 non-null  float64
 1   col_1   100000 non-null  float64
 2   col_2   100000 non-null  float64
 3   col_3   100000 non-null  float64
 4   col_4   100000 non-null  float64
 5   col_5   100000 non-null  float64
 6   col_6   100000 non-null  float64
 7   col_7   100000 non-null  float64
 8   col_8   100000 non-null  float64
 9   col_9   100000 non-null  float64
dtypes: float64(10)
memory usage: 7.6 MB


In [ ]:
df_feather.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   col_0   100000 non-null  float64
 1   col_1   100000 non-null  float64
 2   col_2   100000 non-null  float64
 3   col_3   100000 non-null  float64
 4   col_4   100000 non-null  float64
 5   col_5   100000 non-null  float64
 6   col_6   100000 non-null  float64
 7   col_7   100000 non-null  float64
 8   col_8   100000 non-null  float64
 9   col_9   100000 non-null  float64
dtypes: float64(10)
memory usage: 7.6 MB


## Testing with categorical data.
Now let us create data with both the categorical and numerical values and check the performances of different file formats.

In [ ]:
!pip install lorem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Prepare Data

In [ ]:
from lorem import sentence

words = np.array(sentence().strip().lower().replace(".", " ").split())

np.random.seed(0)  
n = 5000000
df = pd.DataFrame(np.c_[np.random.randn(n, 5),
                  np.random.randint(0,10,(n, 2)),
                  np.random.randint(0,1,(n, 2)),
np.array([np.random.choice(words) for i in range(n)])] , 
columns=list('ABCDEFGHIJ'))

df=df.astype(dtype={'A': float, 'B': float, 'C': float, 'D': float, 'E': float, 'F': int, 'G': int, 'H': int, 'I': int, 'J': str}, copy=True)
df.loc[::10, 'A'] = np.NaN
len(df)

5000000

### Time to write

In [ ]:
%time df.to_csv('test_big.csv', index=False)

CPU times: user 44 s, sys: 921 ms, total: 44.9 s
Wall time: 45 s


In [ ]:
%time df.to_hdf('test_big.h5', key='root', index=False)

CPU times: user 2.21 s, sys: 828 ms, total: 3.03 s
Wall time: 3.72 s


In [ ]:
%time df.to_parquet('test_big.parquet', index=False)

CPU times: user 1.38 s, sys: 336 ms, total: 1.72 s
Wall time: 1.69 s


In [ ]:
%time df.to_feather('test_big.feather')

CPU times: user 910 ms, sys: 316 ms, total: 1.23 s
Wall time: 1.1 s


### File size on disk

In [ ]:
%%bash
du -sh test_big.*

532M	test_big.csv
244M	test_big.feather
1.3G	test_big.h5
195M	test_big.parquet


### Time to read

In [ ]:
%%time 
df_big_csv=pd.read_csv('test_big.csv')

CPU times: user 5.8 s, sys: 9.08 s, total: 14.9 s
Wall time: 14.9 s


In [ ]:
%%time
df_big_hdf=pd.read_hdf('test_big.h5', key='root')

CPU times: user 881 ms, sys: 579 ms, total: 1.46 s
Wall time: 1.46 s


In [ ]:
%%time
df_big_parquet=pd.read_parquet('test_big.parquet')

CPU times: user 569 ms, sys: 595 ms, total: 1.16 s
Wall time: 862 ms


In [ ]:
%%time
df_big_feather=pd.read_feather('test_big.feather')

CPU times: user 429 ms, sys: 534 ms, total: 964 ms
Wall time: 641 ms


In [ ]:
df_big_parquet.dtypes

A    float64
B    float64
C    float64
D    float64
E    float64
F      int64
G      int64
H      int64
I      int64
J     object
dtype: object

### Memory Usage

In [ ]:
df_big_csv.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   A       float64
 1   B       float64
 2   C       float64
 3   D       float64
 4   E       float64
 5   F       int64  
 6   G       int64  
 7   H       int64  
 8   I       int64  
 9   J       object 
dtypes: float64(5), int64(4), object(1)
memory usage: 644.4 MB


In [ ]:
df_big_hdf.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   A       float64
 1   B       float64
 2   C       float64
 3   D       float64
 4   E       float64
 5   F       int64  
 6   G       int64  
 7   H       int64  
 8   I       int64  
 9   J       object 
dtypes: float64(5), int64(4), object(1)
memory usage: 682.6 MB


In [ ]:
df_big_parquet.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   A       float64
 1   B       float64
 2   C       float64
 3   D       float64
 4   E       float64
 5   F       int64  
 6   G       int64  
 7   H       int64  
 8   I       int64  
 9   J       object 
dtypes: float64(5), int64(4), object(1)
memory usage: 644.4 MB


In [ ]:
df_big_feather.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   A       float64
 1   B       float64
 2   C       float64
 3   D       float64
 4   E       float64
 5   F       int64  
 6   G       int64  
 7   H       int64  
 8   I       int64  
 9   J       object 
dtypes: float64(5), int64(4), object(1)
memory usage: 644.4 MB


We can see in the above cells that when we use raw categorical fields the size of the file gets large in case of feather and hdf. Now let us use the pd.Categorical function and modify the data and check what happens.

In [ ]:
df['J'] = pd.Categorical(df['J'])

In [ ]:
df.dtypes

A     float64
B     float64
C     float64
D     float64
E     float64
F       int64
G       int64
H       int64
I       int64
J    category
dtype: object

### Time to write

In [ ]:
%time df.to_csv('test_big_pd_cat.csv')

CPU times: user 47.8 s, sys: 976 ms, total: 48.7 s
Wall time: 49.7 s


In [ ]:
%time df.to_hdf('test_big_pd_cat.h5', key='root', format='table')

CPU times: user 3.21 s, sys: 655 ms, total: 3.86 s
Wall time: 5.29 s


In [ ]:
%time df.to_parquet('test_big_pd_cat.parquet')

CPU times: user 969 ms, sys: 276 ms, total: 1.25 s
Wall time: 1.31 s


In [ ]:
%time df.to_feather('test_big_pd_cat.feather')

CPU times: user 695 ms, sys: 240 ms, total: 935 ms
Wall time: 754 ms


### File size on disk

In [ ]:
%%bash
du -sh test_big_pd_cat.*

569M	test_big_pd_cat.csv
219M	test_big_pd_cat.feather
517M	test_big_pd_cat.h5
195M	test_big_pd_cat.parquet


### Time to read

In [ ]:
%%time
df_big_pd_cat_csv = pd.read_csv('test_big_pd_cat.csv')

CPU times: user 6.19 s, sys: 1.46 s, total: 7.65 s
Wall time: 7.66 s


In [ ]:
%%time
df_big_pd_cat_hdf = pd.read_hdf('test_big_pd_cat.h5')

CPU times: user 827 ms, sys: 254 ms, total: 1.08 s
Wall time: 1.08 s


In [ ]:
%%time
df_big_pd_cat_parquet = pd.read_parquet('test_big_pd_cat.parquet')

CPU times: user 411 ms, sys: 577 ms, total: 988 ms
Wall time: 695 ms


In [ ]:
%%time
df_big_pd_cat_feather = pd.read_feather('test_big_pd_cat.feather')

CPU times: user 321 ms, sys: 406 ms, total: 726 ms
Wall time: 492 ms


## Feather or Parquet
1. Parquet format is designed for long-term storage, where Arrow is more intended for short term or ephemeral storage because files volume are larger.

2. Parquet is usually more expensive to write than Feather as it features more layers of encoding and compression.

3. Feather is unmodified raw columnar Arrow memory. We will probably add simple compression to Feather in the future.

4. Due to dictionary encoding, RLE encoding, and data page compression, Parquet files will often be much smaller than Feather files

5. Parquet is a standard storage format for analytics that’s supported by Spark. So if you are doing analytics, Parquet is a good option as a reference storage format for query by multiple systems

[Source StackOverflow](https://stackoverflow.com/questions/48083405/what-are-the-differences-between-feather-and-parquet)

## Bibliograpgy
This notebook is inspired from the following link.<br>

[File Format - Python tools for big data](https://pnavaro.github.io/big-data/14-FileFormats.html)